In [1]:
import sys
sys.path.append("D:/Experiment")
from MyKu import processing
from MyKu import training
from MyKu import MHeadAttention
import pandas as pd
from tqdm import tqdm
import pandas as pd
import torchtext
from torchtext.vocab import Vectors
from torchtext.legacy import data
import torch
from torch import nn
import torch.nn.functional as F
from d2l import torch as d2l
from torch.autograd import Variable
from spacy.lang.en import English
from sklearn import metrics

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)


cuda:0


In [2]:
processing.create_Sem2018()

In [2]:
def tokenizer(text):  # create a tokenizer function
    """
    定义分词操作
    """
    return processing.Pre_processing_tweets().tokenize_process(text)


def DataLoader():

    TEXT = data.Field(sequential=True, tokenize=tokenizer,
                      lower=True, include_lengths=True, fix_length=20)
    LABEL = data.Field(sequential=False, use_vocab=False)
    # 假如train.csv文件并不是只有两列，比如1、3列是review和polarity，2列是我们不需要的数据，
    # 那么就要添加一个全是None的元组， fields列表存储的Field的顺序必须和csv文件中每一列的顺序对应，

    train_fields = [(None, None), ('label', LABEL),  ('tweet', TEXT)]
    # train_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    train_data = data.TabularDataset(
        path='D:/Experiment/datasets/SEM2018/train.tsv',
        # path='D:/Experiment/datasets/EXIST2021/train.tsv',
        format='tsv',
        fields=train_fields,
        skip_header=True  # 是否跳过文件的第一行
    )
    test_fields = [(None, None), ('label', LABEL),  ('tweet', TEXT)]
    # test_fields = [(None, None), (None, None), (None, None), (None, None), ('text', TEXT), ('task1', LABEL)]
    test_data = data.TabularDataset(
        path='D:/Experiment/datasets/SEM2018/test.tsv',
        # path='D:/Experiment/datasets/EXIST2021/test.tsv',
        format='tsv',
        fields=test_fields,
        skip_header=True  # 是否跳过文件的第一行
    )
    return train_data, test_data, TEXT, LABEL


In [3]:
train_data, test_data, TEXT, LABEL = DataLoader()

vectors = Vectors(name='glove.6B.300d.txt', cache=processing.EMBEDDING_PATH)

TEXT.build_vocab(train_data,  # 建词表是用训练集建，不要用验证集和测试集
                  max_size=400000, # 单词表容量
                  vectors=vectors, # 还有'glove.840B.300d'已经很多可以选
                  unk_init=torch.Tensor.normal_ # 初始化train_data中不存在预训练词向量词表中的单词
)

In [4]:
train_data, test_data, TEXT, LABEL = DataLoader()

vectors = Vectors(name='glove.6B.300d.txt', cache=processing.EMBEDDING_PATH)

TEXT.build_vocab(train_data,  # 建词表是用训练集建，不要用验证集和测试集
                  max_size=400000, # 单词表容量
                  vectors=vectors, # 还有'glove.840B.300d'已经很多可以选
                  unk_init=torch.Tensor.normal_ # 初始化train_data中不存在预训练词向量词表中的单词
)

In [9]:

class Model(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout, **kwargs):
        super(Model, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.encoder = nn.LSTM(
            embed_size, num_hiddens, num_layers=num_layers, bidirectional=True, dropout=dropout)
        self.decoder = nn.Linear(num_hiddens * 4, 2)

    def forward(self, inputs):
        # input torch.Size([64, 40, 300])
        input = self.embedding(inputs.permute(1, 0))
        self.encoder.flatten_parameters()
        outputs, _ = self.encoder(input)
        output = torch.cat((outputs[:,0,:], outputs[:,-1,:]), dim=1)    # output torch.Size([64, 800])
        outs = self.decoder(output)
        return outs


In [16]:

num_hiddens, num_layers, dropout = 100, 2, 0.7
model = Model(len(TEXT.vocab), 300, num_hiddens, num_layers, dropout)
model.to(DEVICE)
pretrained_embeddings = TEXT.vocab.vectors
# print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(300)
model.embedding.weight.data[PAD_IDX] = torch.zeros(300)


train_iter, test_iter = data.BucketIterator.splits((train_data, test_data), batch_size=64, sort_within_batch=True, sort_key=lambda x: len(x.tweet), device=DEVICE)


In [7]:
def train(model, train_iter, optimizer, loss, epoch):
    model.train()
    epoch_loss = 0
    num_sample = 0
    correct = 0
    for batch in tqdm(train_iter, desc=f"Training Epoch {epoch}", colour='red'):
        optimizer.zero_grad()
        text, text_len = batch.tweet
        label = batch.label
        # text, text_len = batch.text
        # label = batch.task1
        output = model(text)
        pred_y = torch.argmax(output, dim=1)
        correct += torch.sum(pred_y == label)
        l = loss(output, label)
        l.backward()
        epoch_loss += l.item()
        num_sample += len(batch)
        optimizer.step()
    print(
        f'\tTrain Loss: {epoch_loss / num_sample:.3f} | Train Acc: {correct.float() / num_sample* 100:.2f}%')


def test(model, test_iter):
    true_y, pred_y = [], []
    for batch in tqdm(test_iter, desc=f"Testing", colour='green'):
        text, text_len = batch.tweet
        label = batch.label
        # text, text_len = batch.text
        # label = batch.task1
        with torch.no_grad():
            output = model(text)
            pred_y.extend(output.argmax(dim=1).tolist())
            true_y.extend(label.tolist())
    print(metrics.confusion_matrix(true_y, pred_y))
    print(metrics.classification_report(true_y, pred_y))
    print(f'Acc : {metrics.accuracy_score(true_y, pred_y)}\t F1: {metrics.f1_score(true_y, pred_y, average="macro")}')


In [20]:
lr, num_epochs = 0.0001, 20

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss = nn.CrossEntropyLoss()
for epoch in range(1, num_epochs+1):
    train(model, train_iter, optimizer, loss, epoch)
    test(model, test_iter)


Training Epoch 1: 100%|██████████| 60/60 [00:00<00:00, 89.47it/s]


	Train Loss: 0.007 | Train Acc: 74.48%


Testing: 100%|██████████| 13/13 [00:00<00:00, 295.19it/s]


[[292 181]
 [148 163]]
              precision    recall  f1-score   support

           0       0.66      0.62      0.64       473
           1       0.47      0.52      0.50       311

    accuracy                           0.58       784
   macro avg       0.57      0.57      0.57       784
weighted avg       0.59      0.58      0.58       784

Acc : 0.5803571428571429	 F1: 0.5686797153917543


Training Epoch 2: 100%|██████████| 60/60 [00:00<00:00, 90.83it/s]


...

[[346 127]
 [179 132]]
              precision    recall  f1-score   support

           0       0.66      0.73      0.69       473
           1       0.51      0.42      0.46       311

    accuracy                           0.61       784
   macro avg       0.58      0.58      0.58       784
weighted avg       0.60      0.61      0.60       784

Acc : 0.6096938775510204	 F1: 0.5782723341419681


Training Epoch 20: 100%|██████████| 60/60 [00:00<00:00, 94.82it/s]


	Train Loss: 0.007 | Train Acc: 75.66%


Testing: 100%|██████████| 13/13 [00:00<00:00, 309.25it/s]

[[279 194]
 [133 178]]
              precision    recall  f1-score   support

           0       0.68      0.59      0.63       473
           1       0.48      0.57      0.52       311

    accuracy                           0.58       784
   macro avg       0.58      0.58      0.58       784
weighted avg       0.60      0.58      0.59       784

Acc : 0.5829081632653061	 F1: 0.5758691714023376
